In [2]:
import numpy as np
def load_glove_embeddings(path):
    word2vec_dict = {}            
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                word2vec_dict[word] = vector
            except ValueError:                
                continue    
    return word2vec_dict

path = "glove.840B.300d.txt"
word2vec = load_glove_embeddings(path)
print(len(word2vec))


import pickle

with open('w2v.pkl', 'wb') as f:
    pickle.dump(word2vec, f)

2195884


## --- process the fraction of datasets ---

In [4]:
import pandas as pd
namelist = ['cr', 'trec', 'agnews', 'pc', 'yelp', 'kaggle_med', 'cardio', 'bbc', 'sst2','subj']
for name in namelist:
    df = pd.read_csv(f'data/original/{name}/train.csv')
    df = df.sample(frac=0.5, random_state=100)
    df.to_csv(f'data/original/{name}/train_50.csv', index=False)


    

In [2]:
import torch
torch.cuda.is_available()

True